In [ ]:
import requests
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#plt.rc('font', family='DejaVu Sans', serif='Times')
#plt.rc('text', usetex=True)
from pprint import pprint
from datetime import datetime

This API provides access to the JPL/SSD small-body mission design suite. The following operation modes are available:

* Mode A (accessible) - retrieves the list of accessible small bodies based on user-defined constraint.
* Mode Q (query) - retrieves pre-computed mission options to a specific object. Both impulsive and low-thrust gravity-assist mission options are available.
* Mode M (map) - an extension of mode Q for the impulsive case, returns the data required to render a porkchop plot with multiple parameters.
* Mode T (mission extension) - retrieves the list of small bodies that come closest (or within a prescribed distance) to a user-defined orbit during a certain period of time. This is a crude filter for finding potential candidates for mission extensions.

This script emphazise the development of the following mode

* Mode M - In addition to querying the database like in mode Q (ballistic), compute all ballistic mission options to the specified target within certain ranges of launch dates and times of flight.
    * In addition, the values of the x-_y_ axes of the maps are also provided:
        * dep_date - departure dates from Earth (Modified Julian Date), corresponding to the x-axis.
        * tof - times of flight to the target (days), corresponding to the y-axis.
        * If dep_date has m elements and tof has n, then the 2D arrays are of dimension n x m.
        * vinf_dep
        * vinf_arr

[Reference of the API construction, data input and output](https://ssd-api.jpl.nasa.gov/doc/mdesign.html)

# Single example Query 

In [ ]:
url_base = 'https://ssd-api.jpl.nasa.gov/mdesign.api'
asteroid_name = '2012TC4' # designation (provisional or IAU-number) of the desired object (e.g., 2015 AB or 141P or 433).
                         # NOTE: when submitting a des containing a space in your query string, you must 
                         # replace the space with %20, for example 2015%20AB.
mjd0 = 59215            # 21Jan2021 # first launch date to be explored (Modified Julian Date)
span = 365                # duration of the launch-date period to be explored (days)
tof_min = 50          # minimum time of flight to be considered (days)
tof_max = 720       # maximum time of flight to be considered (days)
step = 5            # time step used to advance both the launch date and the time of flight (days). 
                    # The size of the transfer map is limited to 1,500,000 points

sim_lim_points = 1500000 #1.5 millions
if int(span)/int(step) > sim_lim_points:
    print('outside of tool limits') 

url = f'{url_base}?sstr={str(asteroid_name)}&mjd0={str(mjd0)}&span={str(span)}&tof-min={str(tof_min)}&tof-max={str(tof_max)}&step={str(step)}'
r = requests.get(url)

data = r.json()

print(data)

# Functions

In [ ]:
def get_mission_profiles(asteroid_name,mjd0,span,tof_min,tof_max,step):
    # asteroid_name:    designation (provisional or IAU-number) of the desired object (e.g., 2015 AB or 141P or 433).
    #                   NOTE: when submitting a des containing a space in your query string, you must replace the space with %20, for example 2015%20AB
    # mjd0:             first launch date to be explored (Modified Julian Date)
    # span:             duration of the launch-date period to be explored (days)
    # tof-min:          minimum time of flight to be considered (days)
    # tof-max:          maximum time of flight to be considered (days)
    # step:             time step used to advance both the launch date and the time of flight (days). 
                        
    # The size of the transfer map is limited to 1,500,000 points
    sim_lim_points = 1500000 #1.5 millions
    if int(span)/int(step) > sim_lim_points:
        print('outside of tool limits') # TODO return error
    
    # Construction of the HTTP request
    url_base = 'https://ssd-api.jpl.nasa.gov/mdesign.api'
    url = f'{url_base}?sstr={str(asteroid_name)}&mjd0={str(mjd0)}&span={str(span)}&tof-min={str(tof_min)}&tof-max={str(tof_max)}&step={str(step)}'
    r = requests.get(url)
    data = r.json()
    
    # Elaboration of data
    available_missions = len(data['selectedMissions'])
    mission_profiles={};
    mjd01Jan2021 = 59215
    mjd01Jan2048 = 69076
    for mission_id in range(available_missions):
        if (data["selectedMissions"][mission_id][0] > mjd01Jan2021 and data["selectedMissions"][mission_id][1] < mjd01Jan2048):
            sel_profile={"fullname": data["object"]["fullname"],
                      "mjd0": data["selectedMissions"][mission_id][0],
                      "mjdf": data["selectedMissions"][mission_id][1],
                      "tof": data["selectedMissions"][mission_id][9],
                      "vinf_dep": data["selectedMissions"][mission_id][2],
                      "vinf_arr": data["selectedMissions"][mission_id][3],
                      "earth_dist": data["selectedMissions"][mission_id][5],
                      "phase_ang": data["selectedMissions"][mission_id][4], 
                      "elong_arr": data["selectedMissions"][mission_id][6], 
                      "decl_dep": data["selectedMissions"][mission_id][7],
                      "approach_ang": data["selectedMissions"][mission_id][8],      
                     };
        mission_profiles[mission_id]=sel_profile;
    
    # Find min dv mission profile
    mission_profile_min_dv, mp_dv_plot = get_min_dv_mission_profile(mission_profiles);
    
    # Porkchop data
    porkchop_dv, dep_date, tof, pc_plot = get_mission_porkchop(data);
    
    return mission_profiles, porkchop_dv, dep_date, tof, pc_plot, mission_profile_min_dv, mp_dv_plot

In [ ]:
def get_mission_porkchop(data):
    dep_date=data["dep_date"];
    tof=data["tof"];
    
    # Elaboration of data
    m = len(data["dep_date"])
    n = len(data["tof"])
    porkchop_map = np.zeros([n,m]) # porkchop_map[i,j]
    for i in range(n):
        for j in range(m):
            porkchop_map[i,j]=abs(data["vinf_arr"][i][j])+abs(data["vinf_dep"][i][j])
    
    # Porchop Plot
    fig = plt.figure()
    plt.contour(dep_date, tof, porkchop_map, np.linspace(0,50,51), cmap="gnuplot")
    fig.suptitle('Porkchop Plot for '+ data["object"]["fullname"])
    plt.xlabel('$Date_{dep}$ (mjd)')
    plt.ylabel('$ToF$ (d)')                               
    plt.colorbar();
    return porkchop_map, dep_date, tof, fig

In [ ]:
def get_min_dv_mission_profile(mission_profiles):
     # Find the best Mission Profile
    dv = np.zeros(len(mission_profiles))
    for profile in mission_profiles:
        dv[profile] = mission_profiles[profile]['vinf_dep'] + mission_profiles[profile]['vinf_arr']

    index = np.linspace(0,len(dv)-1,len(dv))
    mask = [dv == np.min(dv)]
    index_min = index[mask]
    mission_profile_min_dv = mission_profiles[index_min[0]] # NOTE: there could be more than one best solution, here i took arbitrarly the first
    
    # Plot of the mission profiles and highlight the best one
    fig = plt.figure()
    plt.plot(dv, "*");
    fig.suptitle('Mission Profile dv Distribution for '+ mission_profile_min_dv["fullname"])
    plt.xlabel('$idx$ (-)')
    plt.ylabel('$dv$ (km/s)')
    plt.plot(int(index_min[0]), dv[int(index_min[0])], "r+");
    
    return mission_profile_min_dv, fig

In [ ]:
##### DO NOT RUN NOW, TO CHECK THE USE


# Mode T - Request list of small bodies that come closest (or within a prescribed distance) to a user-specified heliocentric 
# orbit (assumes two-body dynamics). Proxy for easiest-to-reach targets for an extended mission phase.
# example url
# https://ssd-api.jpl.nasa.gov/mdesign.api?ec=0.2056408220896557&qr=0.3074958016246215&tp=2459067.6508400026&
# om=48.30597718083336&w=29.18348714438387&in=7.003733902930839&jd0=2458849.5&jdf=2459132.5&maxout=100&maxdist=0.0010

def get_close_approach_to_asteroid(orb_params,jd0,jdf,n_object_requested,distance_within)
    # orb_params:            array containing the orbital parameters required to run the query
    #     ec:                eccentricity [>0]
    #     qr:                perihelion distance [>0]
    #     tp:                time of perihelion passage (JD)
    #     OM:                longitude of the ascending node (deg) [0,360]
    #     om:                argument of periapsis (deg) [0,360]
    #     incl:              inclination (deg) [0,180]
    # jd0:                   beginning of the requested time span (JD)
    # jdf:                   end of the requested time span (JD). Time span must not be longer than one year
    # n_object_requested:    maximum number of records to be returned
    # distance_within:       ignore objects with distance of closest approach greater than "distance_within" [>0, optional]
    
    # Extraction of inputs
    ec = orb_params[1]
    qr = orb_params[2]
    tp = orb_params[3]
    OM = orb_params[4]
    om = orb_params[5]
    incl = orb_params[6]
    
    # Construction of the HTTP request
    url_base = 'https://ssd-api.jpl.nasa.gov/mdesign.api'
    url = f'{url_base}?ec={str(ec)}&qr={str(qr)}&tp={str(tp)}&om={str(OM)}&w={str(om)}&in={str(incl)}&jd0={str(jd0)}&jdf={str(jdf)}&maxout={str(n_object_requested)}&maxdist={str(distance_within)}'
    r = requests.get(url)
    data = r.json()
    return data

# Function Call

In [ ]:
req_ast_name = '2012TC4'
req_mjd0 = 59215
req_duration = 365
req_min_tof = 50
req_max_tof = 700
req_step_size = 5

missions_2012tc4, porkchop_dv_2012tc4, dep_date_2012tc4, tof_2012tc4, pc_plot_2012tc4, mp_min_dv_2012tc4, mp_dv_plot_2012tc4 = \
    get_mission_profiles(req_ast_name,req_mjd0,req_duration,req_min_tof,req_max_tof,req_step_size)